In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [ ]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [ ]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i, c in enumerate(world_set)}

In [ ]:
print(vocab)

{'s': 0, 'a': 1, 'r': 2, 'p': 3, 'B': 4, 'b': 5, "'": 6, 't': 7, ',': 8, 'd': 9, 'c': 10, 'y': 11, ' ': 12, 'h': 13, 'g': 14, 'l': 15, 'o': 16, 'k': 17, 'u': 18, 'n': 19, 'm': 20, 'w': 21, 'f': 22, '.': 23, 'e': 24, 'i': 25}


In [ ]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [ ]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.1

In [ ]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+10]
  y_str = sentence[i+1:i+11]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[4, 2, 25, 10, 17, 12, 21, 1, 15, 15]
[2, 25, 10, 17, 12, 21, 1, 15, 15, 0]


In [ ]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [ ]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [ ]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [ ]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 2, 25, 10, 17, 12, 21,  1, 15, 15,  0])


In [ ]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2)

In [ ]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [ ]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

dhdhdhhhhhhhhdhhhhhdhhhdhhhdhhhdhhhhhdhhhdhhdhdhhhdhhhhhhhhhhdhhhhhhddhdhhhhhhhhdhhhhhhhhdhhhhhdhhhhhhhhddhhhddhhddhhdhhhhhhhhhhhhhhhhdhhhhhhddhdhhhhhhhhdhhhhhdhhhhhhhhhhhhhhhhhhddhhhhhhhhhhhhhhhhh
fefllsfllssrleyrleyelefeyrrleyrkekrkllyrlefrkllsllllleyelslellslelelefrlsrllssrleyrllllelefefeyrllllslllelelrsllllleyeyrkekelllrllsfeleslelelefrlsrllssrleyrleyelefeyrllsllllslllllllllrllrlllyllylll
a h a e a    ea    a    e      e  a e   e ae a  h     a  a  a a aae    e a e    ea    a    aa a t       a   t aa  a a   t      h  a e aa   e    e a e    ea    a    a t       ae  e a t  a           
 t  t  t                                                                                                                                                                                             
 s  s  re          e                                  e         e                     e                                                                        e                                     
    e  te 

In [ ]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want things."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

어린왕자-"어떤 별에 사는 꽃을 좋아한다면
밤에 하늘을 쳐다보는 게 즐거울 거야.
어느 별이나 다 꽃이 필테니까."

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 17.9 MB/s eta 0:00:00


In [ ]:
sentence = ("어떤 별에 사는 꽃을 좋아한다. "
            "밤에 하늘을 쳐다보는 게 즐거울 것이다. "
            "어느 별이나 다 꽃이 피기 때문이다. ")

In [ ]:
from konlpy.tag import Okt
tokenizer = Okt()
word = tokenizer.morphs(sentence)

In [ ]:
word

['어떤',
 '별',
 '에',
 '사는',
 '꽃',
 '을',
 '좋아한다',
 '.',
 '밤',
 '에',
 '하늘',
 '을',
 '쳐다보는',
 '게',
 '즐거울',
 '것',
 '이다',
 '.',
 '어느',
 '별',
 '이나',
 '다',
 '꽃',
 '이',
 '피기',
 '때문',
 '이다',
 '.']

In [ ]:
vocab=list(set(word))
print(vocab)

['게', '별', '때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에', '을', '이나', '것', '다', '쳐다보는', '꽃', '피기']


In [ ]:
word2index = {tkn: i for i, tkn in enumerate(vocab)}
print(word2index)

{'게': 0, '별': 1, '때문': 2, '즐거울': 3, '어느': 4, '밤': 5, '좋아한다': 6, '하늘': 7, '어떤': 8, '사는': 9, '이': 10, '.': 11, '이다': 12, '에': 13, '을': 14, '이나': 15, '것': 16, '다': 17, '쳐다보는': 18, '꽃': 19, '피기': 20}


In [ ]:
vocab_size=len(word2index)
vocab_size

21

In [ ]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{0: '게', 1: '별', 2: '때문', 3: '즐거울', 4: '어느', 5: '밤', 6: '좋아한다', 7: '하늘', 8: '어떤', 9: '사는', 10: '이', 11: '.', 12: '이다', 13: '에', 14: '을', 15: '이나', 16: '것', 17: '다', 18: '쳐다보는', 19: '꽃', 20: '피기'}


In [ ]:
def build_data(word, word2index):
    encoded = [word2index[token] for token in word]  
    input_seq, label_seq = encoded[:-1], encoded[1:] 
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) 
    return input_seq, label_seq

In [ ]:
X, Y = build_data(word, word2index)

In [ ]:
print(X)

tensor([[ 8,  1, 13,  9, 19, 14,  6, 11,  5, 13,  7, 14, 18,  0,  3, 16, 12, 11,
          4,  1, 15, 17, 19, 10, 20,  2, 12]])


In [ ]:
print(Y)

tensor([[ 1, 13,  9, 19, 14,  6, 11,  5, 13,  7, 14, 18,  0,  3, 16, 12, 11,  4,
          1, 15, 17, 19, 10, 20,  2, 12, 11]])


In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) 

    def forward(self, x):
        output = self.embedding_layer(x)
        output, hidden = self.rnn_layer(output)
        output = self.linear(output)
        return output.view(-1, output.size(2))

In [ ]:
vocab_size = len(word2index)  
input_size = 5  
hidden_size = 20  

In [ ]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params=model.parameters())

In [ ]:
output = model(X)
print(output)

tensor([[-2.7028e-01, -3.0536e-01,  2.3361e-02,  1.4059e-01, -4.3671e-01,
         -9.8117e-02, -2.2294e-01, -2.6260e-01,  3.2985e-01,  2.8858e-01,
          3.4434e-01,  5.7208e-02,  1.2566e-01,  1.2429e-01,  1.3854e-01,
         -1.2091e-01, -1.2504e-01, -3.4621e-01, -8.7800e-02, -1.3401e-01,
         -1.6946e-02],
        [-2.5868e-01, -2.3050e-01,  7.1767e-02,  4.3174e-01, -9.4876e-02,
         -2.8475e-01, -1.7081e-02, -2.7850e-01,  3.3905e-01,  2.3602e-01,
          4.8124e-01,  1.1830e-01,  2.7212e-01, -6.7683e-02,  5.6648e-01,
         -1.5716e-01,  1.9822e-01, -2.7959e-01, -5.8905e-02,  4.0438e-02,
         -1.3452e-02],
        [-1.2611e-01,  3.2429e-01,  9.3503e-02, -1.4390e-01, -5.0301e-01,
         -2.1652e-03, -1.2540e-01, -5.8548e-01,  6.8160e-01,  2.2887e-01,
          4.1531e-02, -2.7737e-01, -2.1237e-01, -1.9363e-01,  9.6809e-02,
          8.6490e-02, -2.4485e-01, -4.5793e-01, -8.2795e-02, -5.8530e-02,
          1.9544e-02],
        [-9.6225e-03, -3.5054e-02,  1.9974e

In [ ]:
decode = lambda y: [index2word.get(x) for x in y]

In [ ]:
for step in range(201):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 3.0875 
Repeat 이 을 어떤 을 을 쳐다보는 때문 쳐다보는 을 어떤 을 사는 때문 을 어떤 을 을 별 사는 이 을 이 을 즐거울 때문 을 을

[41/201] 2.6681 
Repeat 사는 을 별 을 . 좋아한다 . 별 이다 사는 을 쳐다보는 . 을 . 이다 . 별 때문 . . 이 . 꽃 . . .

[81/201] 2.1478 
Repeat 사는 꽃 사는 꽃 . 좋아한다 . 밤 이다 하늘 을 쳐다보는 게 즐거울 . 꽃 . 어느 별 . 다 꽃 . 꽃 때문 이다 .

[121/201] 1.5778 
Repeat 사는 꽃 사는 꽃 . 좋아한다 . 밤 이다 하늘 을 쳐다보는 게 즐거울 것 꽃 . 어느 별 . 다 꽃 이 꽃 때문 이다 .

[161/201] 1.0894 
Repeat 별 꽃 사는 꽃 을 좋아한다 . 밤 에 하늘 을 쳐다보는 게 즐거울 것 이다 . 어느 별 . 다 꽃 이 피기 때문 이다 .

[201/201] 0.7205 
Repeat 별 에 사는 꽃 을 좋아한다 . 밤 에 하늘 을 쳐다보는 게 즐거울 것 이다 . 어느 별 이나 다 꽃 이 피기 때문 이다 .



task1 그대로

In [ ]:
hidden_size = vocab_size 
sequence_length = 10
learning_rate = 0.01

In [ ]:
x_data = []
y_data = []

for i in range(0, len(vocab) - sequence_length):
  x_str = vocab[i:i+sequence_length]
  y_str = vocab[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  x_data.append([word2index[c] for c in x_str])
  y_data.append([word2index[d] for d in y_str])

0 ['게', '별', '때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는'] -> ['별', '때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이']
1 ['별', '때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이'] -> ['때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.']
2 ['때문', '즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.'] -> ['즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다']
3 ['즐거울', '어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다'] -> ['어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에']
4 ['어느', '밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에'] -> ['밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에', '을']
5 ['밤', '좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에', '을'] -> ['좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에', '을', '이나']
6 ['좋아한다', '하늘', '어떤', '사는', '이', '.', '이다', '에', '을', '이나'] -> ['하늘', '어떤', '사는', '이', '.', '이다', '에', '을', '이나', '것']
7 ['하늘', '어떤', '사는', '이', '.', '이다', '에', '을', '이나', '것'] -> ['어떤', '사는', '이', '.', '이다', '에', '을', '이나', '것', '다']
8 ['어떤', '사는',

In [ ]:
print(x_data[0])
print(y_data[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
x_data

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 [6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 [9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([11, 10, 21])
레이블의 크기 : torch.Size([11, 10])


In [ ]:
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 

In [ ]:
print(Y[0])

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])


In [ ]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
outputs = net(X)
print(outputs.shape)

torch.Size([11, 10, 21])


In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: 
            predict_str += ''.join([vocab[t] for t in result])
        else: 
            predict_str += vocab[result[-1]]

    print(predict_str)

것것것것것다다것이것것이나것이것것이것것것
이사는이다사는사는다다이사는이사는이사는이이다사는이사는이이
이이다이다이다이다이다이다이다사는이사는이다사는이이다사는이다사는이다사는
이이다이다이다이다이다이다이다사는이사는이다사는이다이다이다이다사는이다사는
이이다이다이다이다이다이다이다사는이사는이다사는이다이다이다이다사는이다사는
이이다이다이다이다이다이다이다사는이사는이다에이이다이다이다이이다사는
이이다이다이다에이다.어떤사는이.이다에이이다에에이이다사는
이이다이다이다에.하늘어떤사는이.이다에을이다것을이이다사는
이이다이다에을좋아한다하늘어떤사는이.이다에을것것을을어떤사는
이이다이다에을좋아한다하늘어떤사는이.이다에을것것을별어떤사는
이이다이다에을좋아한다하늘어떤사는이.이다에을것것을별어떤좋아한다
이이다이다어떤을좋아한다하늘어떤사는이.이다에을것것별별별좋아한다
이사는이다에을좋아한다하늘어떤사는이.이다에을것것별별별피기
이사는즐거울에밤좋아한다하늘어떤사는이.이다에을것것별별별피기
이을즐거울어떤밤좋아한다하늘어떤사는이.이다에을것것별별별피기
이좋아한다즐거울어떤밤좋아한다하늘어떤사는이.이다에을것것다쳐다보는꽃피기
이좋아한다즐거울어떤밤좋아한다하늘어떤사는이.이다에을것것다쳐다보는꽃피기
이좋아한다즐거울어떤밤좋아한다하늘어떤사는이.이다에을것것다쳐다보는꽃피기
이좋아한다즐거울어떤밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
이좋아한다즐거울어떤밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
이좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
이좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
을좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
을좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
을좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
이좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
이좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
다좋아한다즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기
다좋

In [ ]:
predict_str

'별때문즐거울어느밤좋아한다하늘어떤사는이.이다에을이나것다쳐다보는꽃피기'